In [2]:
clear all

In [3]:
!pip install polars

  Using cached polars-1.3.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
Using cached polars-1.3.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31.2 MB)


In [4]:
!pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)


In [5]:
!pip install pytorchtools

  Using cached pytorchtools-0.0.2-py2.py3-none-any.whl.metadata (2.2 kB)
Using cached pytorchtools-0.0.2-py2.py3-none-any.whl (3.1 kB)


In [6]:
import polars as pl
import cv2
import glob
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.nn import Module
# from torchsummary import summary
from torchvision import models
import torch.nn.functional as F
from datetime import datetime
import os
from tqdm import tqdm
# import zipfile
# import shutil
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.profiler import profile, record_function, ProfilerActivity

In [5]:
# Go requesting all of my pictures

In [7]:
dfs=[pl.read_csv(filename) for filename in glob.glob('/home/jovyan/my_work/Egypto/dfs/dfs/df_*.csv')]
df=pl.concat(dfs)
df.head()

filenames,classes,bb
str,str,str
"""drive/MyDrive/cropped_pictures…","""""",""""""
"""drive/MyDrive/cropped_pictures…","""2""","""72.0,37.0,236.23839330673218,1…"
"""drive/MyDrive/cropped_pictures…","""""",""""""
"""drive/MyDrive/cropped_pictures…","""0""","""0.0,0.0,225.8830212788887,409.…"
"""drive/MyDrive/cropped_pictures…","""""",""""""


In [8]:
df=df.with_columns(pl.col('filenames').str.split('/').list.last())
df.head()

filenames,classes,bb
str,str,str
"""425942001_300_300_243_543_empt…","""""",""""""
"""425942001_300_300_243_543_cara…","""2""","""72.0,37.0,236.23839330673218,1…"
"""420886001_300_300_1028_1328_em…","""""",""""""
"""420886001_300_300_1028_1328_ca…","""0""","""0.0,0.0,225.8830212788887,409.…"
"""423879001_300_300_814_1114_emp…","""""",""""""


In [9]:
all_images=os.listdir('/home/jovyan/my_work/Egypto/Images')
existing_files_series = pl.Series(all_images)


In [10]:
df=df.with_columns(pl.col('filenames').is_in(all_images).alias('new_image')).filter(pl.col('new_image'))

In [11]:
df=df.drop('new_image')
df.head()

filenames,classes,bb
str,str,str
"""425942001_300_300_243_543_empt…","""""",""""""
"""425942001_300_300_243_543_cara…","""2""","""72.0,37.0,236.23839330673218,1…"
"""420886001_300_300_1028_1328_em…","""""",""""""
"""420886001_300_300_1028_1328_ca…","""0""","""0.0,0.0,225.8830212788887,409.…"
"""423879001_300_300_814_1114_emp…","""""",""""""


In [12]:
df=df.with_columns(pl.col('filenames').str.split('_').list.last().str.split('.').list.first().alias('empty')).filter(pl.col('empty')=='caractere').drop('empty')

We're checkin for duplicates

In [21]:
df['filenames'][0]

'425942001_300_300_243_543_caractere.png'

In [ ]:
df['filenames'].value_counts(

).filter(pl.col('count')>1
         ).with_columns(pl.col('filenames').str.split('_').list.last().str.split('.').list.first().alias('caractere_or_empty')
         ).filter(pl.col('caractere_or_empty')=='caractere').shape

(103, 3)

droping the duplicates and keeping the last one ( since the photos were probably overwritten because they were assigned the same same )

In [13]:
df=df.unique(subset='filenames', keep='last')

In [43]:
df=df[1:,:]

In [14]:
df.shape

(34241, 3)

## Customization du dataset [link here](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [ ]:
df['bb'][0]

''

In [14]:
class CustomDataset(Dataset):
  """
  This Class creates tensor for each value in a dataframe.
  """
  def __init__(self, df, transform:bool):
    """
      df: dataframe with the annotations
    root_dir: directory with all the images
    transform: Optionnal transform to apply
    """

    self.df=df
    self.transform=transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self,idx):
    # On rajoute le transform, voir ce que ça fait sur les résultats
    img_name=self.df['filenames'][idx]
    image_file=os.path.join('/home/jovyan/my_work/Egypto/Images',img_name)
    image=cv2.imread(image_file)
    image=1.6*image - 50
    image=image/255

    if image is None:
      raise FileNotFoundError(f"Image {img_name} not found")
    if self.transform:
      image=self.transform(image)
      #On verrifie que 'bb' et 'classes' sont non vides sinon ca risque de buguer
    if image is not None:
      image = torch.tensor(image, dtype=torch.float32).permute(2,0,1)
    bb_str=self.df['bb'][idx]
    if bb_str:
      bb=list(map(float,self.df['bb'][idx].split(',')))
      target=torch.tensor(bb,dtype=torch.float32)
    else:
      target=torch.tensor([0,0,300,300])
    classe_str=self.df['classes'][idx]
    if classe_str:
        label_map = {'0': 0, '1': 1, '2': 2}
        label = torch.tensor(label_map.get(classe_str, -1), dtype=torch.float32)
    else:
      label = torch.tensor(3, dtype=torch.float32)

    return image, target, label


In [15]:
def custom_collate_fn(batch):
  """
  This function collates the tensor for them to have the same size

  batch: batch in the dataloader
  """
  images, targets, labels = zip(*batch)

    # Convert images from numpy arrays to torch tensors if needed
  images = [torch.tensor(image) if isinstance(image, np.ndarray) else image for image in images]
  images = torch.stack(images, 0)
  bbox= [torch.tensor(t) if isinstance(t, np.ndarray) else t for t in targets]
  bbox_stacked=torch.stack(bbox,0)
  cls= [torch.tensor(c) if isinstance(c, np.ndarray) else c for c in labels]
  cls_stacked=torch.stack(cls,0)

  return images, bbox_stacked, labels

In [16]:
train, test= train_test_split(df, test_size=0.3, random_state=42, shuffle=True, stratify=df['classes'])

penser à changer la typo en snake case et pas en camel_case ( pour les classes )

In [17]:
dataset_train=CustomDataset(train, transform=False )
dataset_test=CustomDataset(test, transform=False)
dataloader_train=DataLoader(dataset_train, batch_size=16, shuffle=True, collate_fn=custom_collate_fn, num_workers=4)
dataloader_test=DataLoader(dataset_test, batch_size=32, shuffle=True, collate_fn=custom_collate_fn, num_workers=4)

On se retrouve avec un problème:
Les tensor n'acceptent pas des valeurs vides.
Donc soit on les remplacent par des valeurs qui ne veulent rien dire, soit on les supprime

In [26]:
for batch in dataloader_train:
    print(batch[1][0])
    break


tensor([140.0000,  37.0000, 256.0008, 213.7068])


In [18]:
class VGG16(nn.Module):
  """_summary_

  This is the model for computer vision
  """
  def __init__(self, num_classes=3, num_bbox_coords=4):
    super(VGG16, self).__init__()
    self.layer1= nn.Sequential(nn.Conv2d(3,64, (3,3)),
                               nn.BatchNorm2d(64),
                               nn.ReLU())
    self.layer2=nn.Sequential(nn.Conv2d(64,64, (3,3)),
                              nn.BatchNorm2d(64),
                              nn.ReLU(),
                              nn.MaxPool2d((2,2), stride=2))
    self.layer3= nn.Sequential(nn.Conv2d(64,128, (3,3)),
                               nn.BatchNorm2d(128),
                               nn.ReLU())
    self.layer4= nn.Sequential(nn.Conv2d(128,128,(3,3)),
                               nn.BatchNorm2d(128),
                               nn.ReLU(),
                               nn.MaxPool2d((2,2), stride=2))
    self.layer5= nn.Sequential(nn.Conv2d(128,256,(3,3)),
                               nn.BatchNorm2d(256),
                               nn.ReLU())
    self.layer6=nn.Sequential(nn.Conv2d(256,256, (3,3)),
                              nn.BatchNorm2d(256),
                              nn.ReLU())
    self.layer7=nn.Sequential(nn.Conv2d(256,256, (3,3)),
                              nn.BatchNorm2d(256),
                              nn.ReLU(),
                              nn.MaxPool2d((2,2), stride=2, ceil_mode=True))
    self.layer8=nn.Sequential(nn.Conv2d(256,512, (3,3)),
                              nn.BatchNorm2d(512),
                              nn.ReLU())
    self.layer9=nn.Sequential(nn.Conv2d(512,512, (3,3)),
                              nn.BatchNorm2d(512),
                              nn.ReLU(),
                             nn.MaxPool2d((2,2),stride=2))
    self.layer10=nn.Sequential(nn.Conv2d(512,512, (3,3)),
                              nn.BatchNorm2d(512),
                              nn.ReLU(),
                               nn.MaxPool2d((2,2), stride=2))
    self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc=nn.Sequential(nn.Dropout(0.6),
                          nn.Linear(512, 4096),
                          nn.ReLU())
    self.fc1=nn.Sequential(nn.Dropout(0.6),
                           nn.Linear(4096, 4096),
                           nn.ReLU())
    self.fc_bbox=nn.Linear(4096,num_bbox_coords)
    self.fc_class=nn.Linear(4096, num_classes)



  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=self.layer3(out)
    out=self.layer4(out)
    out=self.layer5(out)
    out=self.layer6(out)
    out=self.layer7(out)
    out=self.layer8(out)
    out=self.layer9(out)
    out=self.layer10(out)
    out=self.global_avg_pool(out)
    out=out.view(out.size(0), -1)
    out=self.fc(out)
    out=self.fc1(out)
    out_bbox=self.fc_bbox(out)
    out_class=self.fc_class(out)
    return out_bbox, out_class

labels=['class0', 'class1', 'class2']
module=VGG16(num_classes=len(labels)).to('cuda')
#summary(module, input_size=(3,300,300))

In [19]:
class EarlyStopping:
    """Class for Early Stopping the model if it stagnates
    """
    def __init__(self,patience=3, verbose=False, delta=0):
        self.patience=patience
        self.verbose=verbose
        self.counter=0
        self.best_score=None
        self.early_stop=False
        self.val_loss_min=np.Inf
        self.delta=delta
    
    def __call__(self,val_loss,model):
        score=-val_loss
        if self.best_score is None:
            self.best_score=score
        elif score<self.best_score + self.delta:
            self.counter+=1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter>=self.patience:
                self.early_stop=True
        else:
            self.best_score=score
            self.counter=0
            

# On prend la même loss et le même optimizer qu'ici

In [20]:
loss_fn=torch.nn.CrossEntropyLoss()

In [21]:
#optimizer=optim.SGD(module.parameters(), lr=0.01, momentum=0.5)
optimizer=torch.optim.Adam(module.parameters(),lr=0.001, weight_decay=1e-4)
#scheduler=lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [22]:
import torch.nn.functional as F

def custom_loss_fn(bbox_outputs, class_outputs, bbox_targets, class_number):
  """This function calculates the loss between the targets and the outputs of the model.
  Parameters
  ----------
  bbox_outputs : _torch tensor_
      _bounding box outputs of the model_
  class_outputs : _torch tensor_
      _class outputs of the model_
  bbox_targets : _torch tensor_
      _bounding box targets_
  class_number : _torch tensor_
      _class targets_

  Returns the loss
  
  _torch tensors_
      _the total loss, the loss for the bounding boxes and the loss for the classes_
  """
    bbox_targets = torch.stack([b for b in bbox_targets])
    if class_outputs.dtype != torch.float:
      class_outputs = class_outputs.float()

    bbox_loss = F.smooth_l1_loss(bbox_outputs, bbox_targets, reduction='mean')
    class_loss = loss_fn(class_outputs, class_number)
    total_loss = bbox_loss + class_loss
    return total_loss, bbox_loss, class_loss


In [29]:
def train_one_epoch(epoch_index, tb_writer,total_correct, total_samples):
    """This function trains one epoch

    Parameters
    ----------
    epoch_index : _int_
    tb_writer : _path_
        _path for writing the model advances_
    total_correct : _int_
        _number of correct predictions for classes_
    total_samples : _int_

    Returns the train loss after 1000 batchs
    -------
    """
    running_loss = 0
    last_loss = 0

    for i, (image, bbox, classe) in enumerate(dataloader_train):
        # Putting each targets into the same device
        inputs=image.to("cuda")
        bbox_targets = [b.to("cuda") for b in bbox]
        classe=np.asarray(classe)
        class_targets=torch.tensor(classe,dtype=torch.int64).to("cuda")
        # Calculating the losses with the outputs
        bbox_outputs, class_outputs = module(inputs)
        _,predictions=torch.max(class_outputs,1)
        total_correct+=(predictions==class_targets).sum().item()
        total_samples+=class_targets.size(0)
        loss, bbox_loss, class_loss = custom_loss_fn(bbox_outputs, class_outputs, bbox_targets, class_targets)
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        #Observing the advances after 1000 batchs
        if i % 1000 == 999:
            last_loss = running_loss / 1000
            print('batch {} loss: {}'.format(i+1, last_loss))
            tb_x = epoch_index * len(dataloader_train) + i + 1
            #tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0
            accuracy=100*total_correct/total_samples
            # On peut essayer de mesurer la distance entre ce qui est prédit et la vraie bounding box 
            distance=bbox_targets[-1]-bbox_outputs[-1]
            print('distance:',distance)
            print('batch{} acc: {}'.format(i+1, accuracy))
            print(bbox_loss, class_loss)
    #scheduler.step()
    return last_loss


In [30]:
early_stopping=EarlyStopping()
timestamp=datetime.now().strftime('%Y%m%d_%H%M%S')
log_file='log_trains_1'
#os.makedirs(log_file)
writer=SummaryWriter(log_file.format(timestamp))
epoch_number=0
EPOCHS=30
best_v_loss=1
for epoch in tqdm(range(EPOCHS)):
  total_correct=0
  total_samples=0
  total_v_correct=0
  total_v_samples=0    
  print('EPOCH{}'.format(epoch_number + 1))
  module.train(True)
  #Allowing to clear space for better memory management
  torch.cuda.empty_cache()
  avg_loss=train_one_epoch(epoch_number,writer, total_correct, total_samples)
  torch.cuda.empty_cache()
  #print(f"everything works for avg_loss")

  running_vloss=0
  module.eval()
#Evaluating the model for each epoch
  with torch.no_grad():
      
    for i,(vimages, vbbox, vclass) in enumerate(dataloader_test):
      vinputs= vimages.to("cuda")
      vbbox_target=[b.to("cuda") for b in vbbox]
      vclass_targets=np.asarray(vclass)
      vclass_target=torch.tensor(vclass_targets, dtype=torch.int64).to("cuda")
      voutputs_bbox, voutputs_class=module(vinputs)
      _,v_predictions=torch.max(voutputs_class,1)
      total_v_correct+=(v_predictions==vclass_target).sum().item()
      total_v_samples+=vclass_target.size(0)
      vloss, vbbox_loss, vclasse_loss =custom_loss_fn(voutputs_bbox, voutputs_class, vbbox_target, vclass_target)
      running_vloss+=vloss
 #Priting the accuracy 
  v_accuracy=100*total_v_correct/total_v_samples
  print(f'validation acc: {v_accuracy}')
#Printing train losses and validation losses
  avg_vloss=running_vloss/(i+1)
  print('LOSS train {} valid {}'.format(avg_loss, avg_vloss ))
  writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
  writer.flush()
  early_stopping(avg_vloss, module)
    # Track best performance, and save the model's state
  if avg_vloss < best_v_loss:
      best_vloss = avg_vloss
      model_path = 'model_{}_{}'.format(timestamp, epoch_number)
      torch.save(module.state_dict(), model_path)

  epoch_number += 1


  0%|          | 0/30 [00:00<?, ?it/s]

EPOCH1
batch 1000 loss: 37.615050241470335
distance: tensor([-109.5843,   -4.6337,  -26.2985,   30.1387], device='cuda:0',
       grad_fn=<SubBackward0>)
batch1000 acc: 35.1375
tensor(37.1565, device='cuda:0', grad_fn=<SmoothL1LossBackward0>) tensor(1.1337, device='cuda:0', grad_fn=<NllLossBackward0>)



  3%|▎         | 1/30 [03:51<1:52:02, 231.82s/it]

validation acc: 39.44320062299231
LOSS train 37.615050241470335 valid 31.65104866027832
EPOCH2
batch 1000 loss: 31.76504083824158
distance: tensor([  8.7744, -37.8248,   8.3383,  65.4574], device='cuda:0',
       grad_fn=<SubBackward0>)
batch1000 acc: 44.1625
tensor(32.6686, device='cuda:0', grad_fn=<SmoothL1LossBackward0>) tensor(1.3324, device='cuda:0', grad_fn=<NllLossBackward0>)



  7%|▋         | 2/30 [07:41<1:47:33, 230.47s/it]

...

validation acc: 99.96106298062884
LOSS train 6.102120569467544 valid 3.453482151031494
EPOCH29
batch 1000 loss: 5.990585672140122
distance: tensor([-8.9651,  0.0800,  5.5604, -0.6575], device='cuda:0',
       grad_fn=<SubBackward0>)
batch1000 acc: 99.325
tensor(5.4739, device='cuda:0', grad_fn=<SmoothL1LossBackward0>) tensor(1.1921e-07, device='cuda:0', grad_fn=<NllLossBackward0>)



 97%|█████████▋| 29/30 [1:51:42<03:51, 231.37s/it]

validation acc: 99.74690937408741
LOSS train 5.990585672140122 valid 2.940260171890259
EPOCH30
batch 1000 loss: 5.941450531244278
distance: tensor([ 1.1582,  0.7421, 24.3809, 17.0001], device='cuda:0',
       grad_fn=<SubBackward0>)
batch1000 acc: 99.26875
tensor(5.9299, device='cuda:0', grad_fn=<SmoothL1LossBackward0>) tensor(0.0005, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 30/30 [1:55:33<00:00, 231.13s/it]

validation acc: 99.98053149031442
LOSS train 5.941450531244278 valid 3.477871894836426


In [35]:
module.state_dict()
torch.save(module.state_dict(), 'module.pt')